In [1]:
from quasinet.qseqtools import load_trained_qnet
from quasinet.qnet import export_qnet_tree
from quasinet.qnet import load_qnet
from tqdm import tqdm

In [2]:
#model=load_trained_qnet('influenza', 'h1n1;na;2009')
import os
import zipfile
with zipfile.ZipFile('../north_h1n1_ha_20.joblib.zip', 'r') as zip_ref:
    zip_ref.extractall('./')
model=load_qnet('./north_h1n1_ha_20.joblib')
os.remove('./north_h1n1_ha_20.joblib')

In [51]:
import numpy as np
TREEPATH='./'
INDEX=np.arange(566)#model.feature_names
#model.feature_names=np.arange(566)
for i in tqdm(INDEX):
    export_qnet_tree(model,i,TREEPATH+str(i)+'.dot',
                     color_alpha=1.5,edge_color='lightgray',
                     labels=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'])

100%|███████████████████████████████████████| 566/566 [00:00<00:00, 1309.20it/s]


In [22]:
allcolors=list(set(list(model.estimators_[63].labels_)+list(model.estimators_[155].labels_)+list(model.estimators_[14].labels_)+list(model.estimators_[223].labels_)))
allcolors.sort()
allcolors

['A', 'D', 'E', 'G', 'I', 'K', 'L', 'Q', 'S', 'T', 'V', 'Y']

In [20]:
def _rgb_to_hex(rgb):
    """Convert RGB to hex for colors.

    Parameters
    ----------
    rgb : 1-d like array
        RGB array of size 3.

    Returns
    -------
    hex_repr : str
        Hexidecimal representation of rgb.
    """

    hex_repr = "#{:02x}{:02x}{:02x}".format(rgb[0], rgb[1], rgb[2])
    return hex_repr

def _color_brew(n, alpha=1.0):
    """Generate n colors with equally spaced hues.

    Parameters
    ----------
    n : int
        The number of colors required.

    alpha : float
        Factor to upscale all the colors.

    Returns
    -------
    color_array : 2d array-like
        Arrat of size (n, 3), where each row is of form (R, G, B), which are 
        components of each color.
    """

    color_array = []

    # initialize saturation & value; calculate chroma & value shift
    s, v = 0.75, 0.9
    c = s * v
    m = v - c

    for h in np.arange(25, 385, 360. / n).astype(int):
        # calculate some intermediate values
        h_bar = h / 60.
        x = c * (1 - abs((h_bar % 2) - 1))
        # initialize RGB with same hue & chroma as our color
        rgb = [(c, x, 0),
               (x, c, 0),
               (0, c, x),
               (0, x, c),
               (x, 0, c),
               (c, 0, x),
               (c, x, 0)]
        r, g, b = rgb[int(h_bar)]

        # shift the initial RGB values to match value and store
        rgb = [(int(255 * (r + m))),
               (int(255 * (g + m))),
               (int(255 * (b + m)))]

        # perform upscaling
        rgb = np.array(rgb) * alpha
        np.putmask(rgb, rgb > 255, 255)

        color_array.append(rgb.astype(int))

    return np.array(color_array)

In [21]:
labels=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
color_brew = _color_brew(len(labels), alpha=1.5)
class_colors = {labels[i]: color_brew[i] for i in range(len(labels))}


In [54]:
COL={x:class_colors[x] for x in allcolors}
STR=''
for i in COL.items():
    STR=STR+'\definecolor{'+i[0]+'col}{RGB}{'
    STR=STR+','.join([str(x) for x in i[1]])+'}xx'

In [55]:
STR.split('xx')

['\\definecolor{Acol}{RGB}{255,193,85}',
 '\\definecolor{Dcol}{RGB}{255,255,85}',
 '\\definecolor{Ecol}{RGB}{255,255,85}',
 '\\definecolor{Gcol}{RGB}{136,255,85}',
 '\\definecolor{Icol}{RGB}{85,255,150}',
 '\\definecolor{Kcol}{RGB}{85,255,255}',
 '\\definecolor{Lcol}{RGB}{85,255,255}',
 '\\definecolor{Qcol}{RGB}{111,85,255}',
 '\\definecolor{Scol}{RGB}{231,85,255}',
 '\\definecolor{Tcol}{RGB}{255,85,255}',
 '\\definecolor{Vcol}{RGB}{255,85,255}',
 '\\definecolor{Ycol}{RGB}{255,85,97}',
 '']

In [144]:
from Bio import SeqIO
IDX=[14,63,155,223]
IDX=[]
H={}
A=[]
input_file='../../raw_data/gisaid/north_h1n1_ha_20.fasta'
fasta_sequences = SeqIO.parse(open(input_file),'fasta')
for fasta in fasta_sequences:
    name, sequence = fasta.description, np.array(list(str(fasta.seq)))
    #name1=name.split('|')[1].split('[')[-1].split('(')[-2:][0]
    name1=name.split('|')[0]
    if sequence[223] in ['G','Y','S']:
        H[name1]=''.join(sequence[213:243])



In [145]:
name

'A/India/Pun-NIV306112/2021_Apr|A_/_H1N1|$SEGMENT_NAME|2021-04-27|EPI1868849|'

In [146]:
for i in H.items():
    print(i[0],i[1])

A/Norway/4776/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Norway/2967/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Ghana/755/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Ghana/64/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Ghana/2252/2020 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Ghana/2080/2020 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Ghana/140/2020 AYVFVETSRYSKMFKPEIATRPKVRDQEGR
A/Ghana/138/2020 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Ghana/119/2020 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Nagaski/8/2020 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Fujian-Fengze/SWL11027/2020 AYVFVETSRYSKMFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1051/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1060/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1211/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1270/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1040/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1448/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1467/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEGR
A/Cote_D'Ivoire/1049/2021 AYVFVGTSRYSKKFKPEIATRPKVRDQEG

array(['A', 'C', 'D', 'E', 'G', 'I', 'L', 'N', 'S', 'T', 'V', 'Y'],
      dtype='<U32')

In [121]:
np.unique(A)

array(['A', 'D', 'E', 'F', 'G', 'I', 'K', 'N', 'P', 'Q', 'R', 'S', 'T',
       'V', 'Y'], dtype='<U32')